# IbPy and Interactive Brokers Features Demonstration

## Learning Outcomes
At the end of this simple workshop, you will be able to 
1. Extract Account and Portfolio Information
2. Placing Orders
3. Request Market Data
4. Obtain Historical Data
5. Access Market Depth Information
6. Download Real Time Bars
7. Extract Executions Information, including commission report

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from IBWrapper import IBWrapper, contract
from ib.ext.EClientSocket import EClientSocket
from ib.ext.ScannerSubscription import ScannerSubscription

In the example to follow, **`callback`** is our `IBWrapper` instantiated. We **receive** information via `callback`.


In the example to follow, **`tws`** is our `EClientSocket` instantiated. We **request** information via `tws`

In [2]:
accountName = "DI246990"
callback = IBWrapper()             # Instantiate IBWrapper. callback 
tws = EClientSocket(callback)      # Instantiate EClientSocket and return data to callback
host = ""
port = 4002
clientId = 6

In [3]:
tws.eConnect(host, port, clientId) # Connect to TWS

Server Version: 76
TWS Time at connection:20180212 20:25:33 SGT
[-1, 2104, 'Market data farm connection is OK:hfarm']
[-1, 2104, 'Market data farm connection is OK:jfarm']
[-1, 2104, 'Market data farm connection is OK:usfuture']
[-1, 2104, 'Market data farm connection is OK:eufarm']
[-1, 2104, 'Market data farm connection is OK:cashfarm']
[-1, 2104, 'Market data farm connection is OK:usopt']
[-1, 2104, 'Market data farm connection is OK:usfarm.us']
[-1, 2104, 'Market data farm connection is OK:usfarm']
[-1, 2106, 'HMDS data farm connection is OK:ilhmds']
[-1, 2106, 'HMDS data farm connection is OK:euhmds']
[-1, 2106, 'HMDS data farm connection is OK:fundfarm']
[-1, 2106, 'HMDS data farm connection is OK:ushmds']


In [4]:
create = contract()                # Instantiate contract class
callback.initiate_variables()

[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2104, 'Market data farm connection is OK:usopt']


Note how the work flow goes. We send a request via the prefix **tws.** followed by the request for the specific type of data after the dot for the information we are interested in. 

For example, we would like to get an update on account time, which required us calling `reqAccountUpdates`. 

We request for info by calling **`tws.reqAccountUpdates`** and the data will be returned via our callback function. In this case **`callback.update_AccountTime`**

*****

# Account and Portfolio
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Account Updates
   * Account Value
   * Portfolio
   * Account Time
2. Account Summary
3. Positions

### Summary of Account and Portfolio 

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqAccountUpdates | updateAccountValue | self.update_AccountValue |
| | updatePortfolio | self.update_Portfolio |
| | updateAccountTime | self.update_AccountTime |
| reqAccountSummary | accountSummary | self.account_Summary |
| reqPositions | position | self.update_Position |

### Sending Account Updates Request
`reqAccountUpdates`

In [5]:
tws.reqAccountUpdates(1, accountName)

#### Obtaining Account Value
`self.update_AccountValue`

In [9]:
pd.DataFrame(callback.update_AccountValue, 
            columns = ['key', 'value', 'currency', 'accountName'])[:10]

,key,value,currency,accountName
0,AccountCode,DI246990,None,DI246990
1,AccountOrGroup,DI246990,BASE,DI246990
2,AccountOrGroup,DI246990,EUR,DI246990
3,AccountOrGroup,DI246990,USD,DI246990
4,AccountReady,true,None,DI246990
5,AccountType,INDIVIDUAL,None,DI246990
6,AccruedCash,123,BASE,DI246990
7,AccruedCash,0,EUR,DI246990
8,AccruedCash,123,USD,DI246990
9,AccruedCash-C,0.00,USD,DI246990


#### Obtaining Portfolio Value
`self.update_Portfolio`

In [10]:
pd.DataFrame(callback.update_Portfolio, 
             columns=['Contract ID','Currency',
                      'Expiry','Include Expired',
                      'Local Symbol','Multiplier',
                      'Primary Exchange','Right',
                      'Security Type','Strike',
                      'Symbol','Trading Class',
                      'Position','Market Price','Market Value',
                      'Average Cost', 'Unrealised PnL', 'Realised PnL', 
                      'Account Name'])[:3]

,Contract ID,Currency,Expiry,Include Expired,Local Symbol,Multiplier,Primary Exchange,Right,Security Type,Strike,Symbol,Trading Class,Position,Market Price,Market Value,Average Cost,Unrealised PnL,Realised PnL,Account Name
0,197397509,EUR,20180615,False,C DBK JUN 18 1785 1,112.0439,DTB,C,OPT,17.85,DBK,DBK.1,2,0.072362,16.22,244.600000,-472.98,0.0,DI246990
1,197397584,EUR,20180615,False,P DBK JUN 18 1785 1,112.0439,DTB,P,OPT,17.85,DBK,DBK.1,-2,4.989651,-1118.12,432.400000,-253.32,0.0,DI246990
2,258973438,USD,20180316,False,ESH8,50,GLOBEX,0,FUT,0.00,ES,ES,3,2649.655762,397448.36,131972.883333,1529.71,0.0,DI246990


#### Obtaining Account Time
`self.update_AccountTime`

In [11]:
callback.update_AccountTime

'20:39'

[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2104, 'Market data farm connection is OK:usopt']


***

### Sending Account Summary Request
`reqAccountSummary`

This function call can only be made when connected to a Financial Advisor (FA) account. Another way to look at this is that if you have more than one account, use this function.

In [12]:
tws.reqAccountSummary(2,"All","NetLiquidation")

#### Obtaining Account Summary
`self.account_Summary`

In [13]:
pd.DataFrame(callback.account_Summary, 
             columns = ['Request_ID','Account','Tag','Value','Curency'])[:2]

,Request_ID,Account,Tag,Value,Curency
0,2,DI246990,NetLiquidation,1139460.61,USD
1,2,DU274370,NetLiquidation,1536054.42,USD


[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usopt']


***

### Sending Position Request
`reqPositions`

This function call request all positions for all accounts. This is more suitable for Financial Advisor. In the following example, I used pandas selection criteria to disply a specific account position.

In [14]:
tws.reqPositions()

#### Obtaining Position
`self.update_Position`

In [15]:
dat = pd.DataFrame(callback.update_Position, 
                   columns=['Account','Contract ID','Currency','Exchange','Expiry',
                            'Include Expired','Local Symbol','Multiplier','Right',
                            'Security Type','Strike','Symbol','Trading Class',
                            'Position','Average Cost'])
dat[dat["Account"] == accountName]

,Account,Contract ID,Currency,Exchange,Expiry,Include Expired,Local Symbol,Multiplier,Right,Security Type,Strike,Symbol,Trading Class,Position,Average Cost
42,DI246990,9408,USD,NYSE,None,False,MCD,None,None,STK,0.00,MCD,MCD,-4,121.264725
43,DI246990,43645865,USD,NASDAQ,None,False,IBKR,None,None,STK,0.00,IBKR,NMS,7,38.837800
44,DI246990,258973438,USD,None,20180316,False,ESH8,50,None,FUT,0.00,ES,ES,3,131972.883333
45,DI246990,279396694,USD,None,20180921,False,ESU8,50,None,FUT,0.00,ES,ES,11,132972.504545
46,DI246990,197397509,EUR,None,20180615,False,C DBK JUN 18 1785 1,112.0439,C,OPT,17.85,DBK,DBK.1,2,244.600000
47,DI246990,197397584,EUR,None,20180615,False,P DBK JUN 18 1785 1,112.0439,P,OPT,17.85,DBK,DBK.1,-2,432.400000
48,DI246990,12087792,USD,None,None,False,EUR.USD,None,None,CASH,0.00,EUR,EUR.USD,150000,1.122282
49,DI246990,208813720,USD,NASDAQ,None,False,GOOG,None,None,STK,0.00,GOOG,NMS,250,965.364000


*****

# Orders
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Open Order
2. Next Valid ID
3. Order Status

### Summary of Orders

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqIds | nextValidId | self.next_ValidId |
| placeOrder | orderStatus | self.order_Status |
| cancelOrder | | |
| reqOpenOrders & reqAllOpenOrders | openOrder | self.open_Order |
| | orderStatus | self.order_Status |
| reqGlobalCancel | | |

#### Demo - Stock Purchase
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`

In [17]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract("GOOG", "STK", "SMART", "USD")
order_info = create.create_order(accountName, "MKT", 150, "BUY")

#### Placing an Order
`placeOrder`

In [18]:
tws.placeOrder(order_id, contract_info, order_info)

#### Checking Order Status
`self.order_Status`

In [19]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,8,PreSubmitted,0,150,0.0,656507437,0,0.0,6,locate


#### Checking on Open Order
`self.open_Order`

In [20]:
callback.open_Order[:1]

[(8,
  <ib.ext.OrderState.OrderState at 0x10ded1940>)]

#### Cancelling Open Order
`cancelOrder`

In [21]:
tws.cancelOrder(order_id)

[8, 202, 'Order Canceled - reason:']


#### Demo
* ** Request Next Valid Id**. `reqIds`
* ** Using Create**. `create`
* ** Placing an Order to purchase Futures**. `placeOrder`

In [23]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201803", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 3, "BUY")
tws.placeOrder(order_id, contract_info, order_info)
print(order_id)

10
[-1, 2104, 'Market data farm connection is OK:usopt']


#### Checking Order Status

In [24]:
pd.DataFrame(callback.order_Status,
             columns = ['orderId', 'status', 'filled', 'remaining', 'avgFillPrice',
                        'permId', 'parentId', 'lastFillPrice', 'clientId', 'whyHeld'])

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld
0,8,PreSubmitted,0,150,0.0,656507437,0,0.0,6,locate
1,8,Cancelled,0,150,0.0,656507437,0,0.0,6,None
2,10,Filled,3,0,2648.5,656507438,0,2648.5,6,None
3,10,Filled,3,0,2648.5,656507438,0,2648.5,6,None
4,10,Filled,3,0,2648.5,656507438,0,2648.5,6,None


IB provided two more methods:
* `reqOpenOrders()` to request any open orders that were placed from this API client.
* `reqAllOpenOrders()` to request all open orders that were placed from all API clients linked to one TWS and also from the TWS.

Each open order will be fed back through `openOrder()` and `orderStatus()` methods.

Finally, use `reqGlobalCancel()` to cancel all open orders globally.

*****

# Market Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Data
   * Tick Price
   * Tick Size
2. Cancel Market Data
3. Calculate Implied Volatility
4. Calculate Option Price

### Summary of Market Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktData | tickPrice  | self.tick_Price |
|  | tickSize | self.tick_Size |
|  | tickOptionComputation  | self.tick_OptionComputation |
|  | tickGeneric | self.tick_Generic |
|  | tickString | self.tick_String |
|  | tickEFP  | self.tick_EFP |
|  | tickSnapshotEnd | self.tickSnapshotEnd_flag |
| cancelMktData | | |
| calculateImpliedVolatility | tickOptionComputation  | self.tick_OptionComputation |
| cancelcalculateImpliedVolatility | | |
| calculateOptionPrice  | tickOptionComputation  | self.tick_OptionComputation |
| cancelCalculateOptionPrice | | |
| reqMktDataType | marketDataType | self.market_DataType |

The method `reqMktDataType` allows you to toggle between receiving real-time or frozen market data.

#### Requesting Market Data
`reqMktData`

In [25]:
contract_info = create.create_contract('AUD', 'CASH', 'IDEALPRO', 'USD')
tickedId = 1002
tws.reqMktData(tickedId, contract_info, "", False)

#### Receiving Tick Price
`self.tick_Price`

In [27]:
tick_data = pd.DataFrame(callback.tick_Price, 
                         columns = ['tickerId', 'field', 'price', 'canAutoExecute'])
tick_type = {0 : "BID SIZE",
             1 : "BID PRICE",
             2 : "ASK PRICE",
             3 : "ASK SIZE",
             4 : "LAST PRICE",
             5 : "LAST SIZE",
             6 : "HIGH",
             7 : "LOW",
             8 : "VOLUME",
             9 : "CLOSE PRICE",
             10 : "BID OPTION COMPUTATION",
             11 : "ASK OPTION COMPUTATION",
             12 : "LAST OPTION COMPUTATION",
             13 : "MODEL OPTION COMPUTATION",
             14 : "OPEN_TICK",
             15 : "LOW 13 WEEK",
             16 : "HIGH 13 WEEK",
             17 : "LOW 26 WEEK",
             18 : "HIGH 26 WEEK",
             19 : "LOW 52 WEEK",
             20 : "HIGH 52 WEEK",
             21 : "AVG VOLUME",
             22 : "OPEN INTEREST",
             23 : "OPTION HISTORICAL VOL",
             24 : "OPTION IMPLIED VOL",
             27 : "OPTION CALL OPEN INTEREST",
             28 : "OPTION PUT OPEN INTEREST",
             29 : "OPTION CALL VOLUME"}
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,price,canAutoExecute,Type
133,1002,1,0.78343,1,BID PRICE
134,1002,1,0.78342,1,BID PRICE
135,1002,1,0.78343,1,BID PRICE
136,1002,1,0.78342,1,BID PRICE
137,1002,1,0.78343,1,BID PRICE
138,1002,1,0.78342,1,BID PRICE
139,1002,1,0.78341,1,BID PRICE
140,1002,1,0.78342,1,BID PRICE
141,1002,2,0.78344,1,ASK PRICE
142,1002,2,0.78345,1,ASK PRICE


#### Receiving Tick Size
`self.tick_Size`

In [28]:
tick_data = pd.DataFrame(callback.tick_Size, 
                         columns = ["tickerId", "field", "size"])
tick_data["Type"] = tick_data["field"].map(tick_type)
tick_data[-10:]

,tickerId,field,size,Type
1356,1002,3,1000000,ASK SIZE
1357,1002,0,3000000,BID SIZE
1358,1002,0,3000000,BID SIZE
1359,1002,0,1000000,BID SIZE
1360,1002,0,1000000,BID SIZE
1361,1002,0,4500000,BID SIZE
1362,1002,0,1000000,BID SIZE
1363,1002,0,1000000,BID SIZE
1364,1002,0,2000000,BID SIZE
1365,1002,0,1000000,BID SIZE


[-1, 1100, 'Connectivity between IB and Trader Workstation has been lost.']
[-1, 2105, 'HMDS data farm connection is broken:ushmds']
[-1, 2103, 'Market data farm connection is broken:secdefus']
[-1, 2103, 'Market data farm connection is broken:usfarm']
[-1, 2103, 'Market data farm connection is broken:jfarm']
[-1, 2103, 'Market data farm connection is broken:cashfarm']
[-1, 2103, 'Market data farm connection is broken:usfarm.us']
[-1, 2103, 'Market data farm connection is broken:hfarm']
[-1, 2103, 'Market data farm connection is broken:usfuture']
[-1, 2105, 'HMDS data farm connection is broken:euhmds']
[-1, 2105, 'HMDS data farm connection is broken:ilhmds']
[-1, 2103, 'Market data farm connection is broken:eufarm']
[-1, 2105, 'HMDS data farm connection is broken:fundfarm']
[-1, 1102, 'Connectivity between IB and Trader Workstation has been restored - data maintained.']
[-1, 2104, 'Market data farm connection is OK:jfarm']
[-1, 2103, 'Market data farm connection is broken:usopt']
[-1, 

#### Calculate Implied Volatility
`calculateImpliedVolatility`

In [29]:
contract_info = create.create_contract(symbol='NFLX 180316C00265000',
                                       secType='OPT', exchange='SMART', 
                                       currency='USD',
                                       right='CALL', 
                                       strike='265', 
                                       expiry='20180316',
                                       multiplier=100, 
                                       tradingClass="NFLX")
tws.calculateImpliedVolatility(tickedId, 
                               contract_info, 
                               6.71, 
                               255)

In [30]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,0.352453,9223372036854775807,6.71,9223372036854775807,9223372036854775807,9223372036854775807,9223372036854775807,255.0


#### Calculate Option Price
`tick_OptionComputation`

In [31]:
tws.calculateOptionPrice(tickedId, 
                         contract_info, 
                         0.35, 
                         255)

In [32]:
pd.DataFrame(callback.tick_OptionComputation,
             columns=["tickerId", "field", "impliedVol", "delta",
                      "optPrice", "pvDividend", "gamma", "vega",
                      "theta", "undPrice"])

,tickerId,field,impliedVol,delta,optPrice,pvDividend,gamma,vega,theta,undPrice
0,1002,53,0.352453,9.223372e+18,6.71000,9223372036854775807,9.223372e+18,9.223372e+18,9223372036854775807,255.0
1,1002,53,0.350000,3.799643e-01,6.63721,9223372036854775807,1.441497e-02,2.962468e-01,9223372036854775807,255.0


#### Cancelling Market Data Stream
`cancelMktData`

In [33]:
tws.cancelMktData(tickedId)

*****

# Historical Data
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Historical Data

### Summary of Historical Data

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqHistoricalData | historicalData  | self.historical_Data |

In [34]:
#contract_Details = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')
contract_Details = create.create_contract('AAPL', 'STK', 'SMART', 'USD')

In [35]:
data_endtime = datetime.now().strftime("%Y%m%d %H:%M:%S")

#### Requesting Historical Data
`reqHistoricalData`

In [36]:
tickerId = 9002
tws.reqHistoricalData(tickerId, 
                      contract_Details, 
                      data_endtime,
                      "1 M", 
                      "1 day", 
                      "BID", 
                      0, 
                      1)

In [37]:
data= pd.DataFrame(callback.historical_Data, 
                   columns = ["reqId", "date", "open",
                              "high", "low", "close", 
                              "volume", "count", "WAP", 
                              "hasGaps"])
data[-10:]

,reqId,date,open,high,low,close,volume,count,WAP,hasGaps
11,9002,20180131,166.21,168.44,165.43,166.80,-1,-1,-1.0,False
12,9002,20180201,166.86,174.26,162.62,173.45,-1,-1,-1.0,False
13,9002,20180202,171.90,172.26,160.00,160.00,-1,-1,-1.0,False
14,9002,20180205,160.00,163.87,155.35,155.40,-1,-1,-1.0,False
15,9002,20180206,155.61,163.71,154.00,162.90,-1,-1,-1.0,False
16,9002,20180207,161.10,163.39,158.75,158.84,-1,-1,-1.0,False
17,9002,20180208,159.80,161.00,155.04,156.20,-1,-1,-1.0,False
18,9002,20180209,156.51,157.88,150.24,156.73,-1,-1,-1.0,False
19,9002,20180212,140.74,158.93,140.74,158.80,-1,-1,-1.0,False
20,9002,finished-20180112 21:38:11-20180212 21:38:11,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.0,False


*****

# Market Depth
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Market Depth

### Summary of Market Depth

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqMktDepth           | updateMktDepth          | self.update_MktDepth |

In [38]:
contract_info = create.create_contract('EUR', 'CASH', 'IDEALPRO', 'USD')

In [39]:
tickerId = 7000
tws.reqMktDepth(tickerId, contract_info, 5)

In [40]:
operation_type = {0 : "Insert",
                  1 : "Update",
                  2 : "Delete",}
side_type = {0 : "Ask",
             1 : "Bid"}

In [41]:
data_mktdepth = pd.DataFrame(callback.update_MktDepth,
                             columns = ["tickerId", "position", 
                                        "operation", "side", 
                                        "price", "size"])
data_mktdepth["operation_type"] = data_mktdepth["operation"].map(operation_type)
data_mktdepth["side_type"] = data_mktdepth["side"].map(side_type)
data_mktdepth[-10:]

,tickerId,position,operation,side,price,size,operation_type,side_type
633,7000,1,1,1,1.22393,7500000,Update,Bid
634,7000,2,1,1,1.22392,6000000,Update,Bid
635,7000,0,1,0,1.22395,3000000,Update,Ask
636,7000,1,1,0,1.22396,1000000,Update,Ask
637,7000,3,1,0,1.22399,2500000,Update,Ask
638,7000,4,1,0,1.22400,1500000,Update,Ask
639,7000,3,1,1,1.22391,3000000,Update,Bid
640,7000,4,1,1,1.22390,3000000,Update,Bid
641,7000,0,1,0,1.22395,2000000,Update,Ask
642,7000,1,1,0,1.22396,2000000,Update,Ask


[-1, 2103, 'Market data farm connection is broken:secdefus']
[7000, 317, 'Market depth data has been RESET. Please empty deep book contents before applying any new entries.']
[-1, 2103, 'Market data farm connection is broken:cashfarm']
[-1, 2103, 'Market data farm connection is broken:eufarm']
[-1, 2104, 'Market data farm connection is OK:eufarm']
[-1, 2103, 'Market data farm connection is broken:secdefus']
[-1, 2104, 'Market data farm connection is OK:secdefus']


*****

# Real Time Bars
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Real Time Bars

Note:
* **barSize**. Only 5 sec bars are supported. 
* **whatToShow**:
   * TRADES
   * BID
   * ASK
   * MIDPOINT
* **useRTH**:
   * 0 = all data
   * 1 = only data within **R**egular **T**rading **H**ours

### Summary of Real Time Bars

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqRealTimeBars           | realtimeBar          | self.real_timeBar |

In [42]:
contract_Details = create.create_contract('GBP', 'CASH', 'IDEALPRO', 'USD')

[-1, 2104, 'Market data farm connection is OK:cashfarm']


In [43]:
tickerId = 10000
tws.reqRealTimeBars(tickerId, 
                    contract_Details, 
                    5, 
                    "MIDPOINT", 
                    0)

[-1, 2106, 'HMDS data farm connection is OK:cashhmds']


In [45]:
pd.DataFrame(callback.real_timeBar, 
             columns = ["reqId", "time", "open", "high", "low", "close", "volume", "wap", "count"])

,reqId,time,open,high,low,close,volume,wap,count
0,10000,1518443460,1.381705,1.381810,1.381705,1.381810,-1,-1.0,-1
1,10000,1518443465,1.381810,1.381810,1.381790,1.381795,-1,-1.0,-1
2,10000,1518443470,1.381795,1.381805,1.381790,1.381795,-1,-1.0,-1
3,10000,1518443475,1.381795,1.381845,1.381790,1.381840,-1,-1.0,-1
4,10000,1518443480,1.381840,1.381860,1.381825,1.381855,-1,-1.0,-1
5,10000,1518443485,1.381855,1.381855,1.381815,1.381845,-1,-1.0,-1


*****

# Executions
### Learning Outcomes
For this section, you will learn how you can obtain the following information:
1. Request Executions

### Summary of Executions

| Request Call | Functions Utilised | Data Stored in |
| --- | --- | --- |
| reqExecutions           | execDetails          | self.exec_Details_reqId |
| | | self.exec_Details_contract |
| | | self.exec_Details_execution |
| | execDetailsEnd |self.exec_DetailsEnd_flag |
| | commissionReport | self.commission_Report |

In [47]:
tws.reqIds(1)
order_id = callback.next_ValidId + 1
contract_info = create.create_contract(symbol = "ES", secType = "FUT", 
                                       exchange = "GLOBEX", currency = "USD", 
                                       right = None, strike = None,
                                       expiry = "201809", multiplier=None,
                                       tradingClass=None)
order_info = create.create_order(accountName, "MKT", 11, "BUY")
tws.placeOrder(order_id, contract_info, order_info)

In [48]:
tws.reqExecutions(3050, create.exec_filter(clientId, accountName, contract_info))

In [49]:
callback.exec_Details_reqId

3050

In [50]:
callback.exec_Details_contract.__dict__

{'m_conId': 279396694,
 'm_currency': 'USD',
 'm_exchange': 'GLOBEX',
 'm_expiry': '20180921',
 'm_includeExpired': False,
 'm_localSymbol': 'ESU8',
 'm_multiplier': '50',
 'm_right': None,
 'm_secType': 'FUT',
 'm_strike': 0.0,
 'm_symbol': 'ES',
 'm_tradingClass': 'ES'}

In [51]:
callback.exec_Details_execution.__dict__

{'m_acctNumber': 'DI246990',
 'm_avgPrice': 2659.25,
 'm_clientId': 6,
 'm_cumQty': 11,
 'm_evMultiplier': 0,
 'm_evRule': None,
 'm_exchange': 'GLOBEX',
 'm_execId': '0001f4e5.5a811f33.01.01',
 'm_liquidation': 0,
 'm_orderId': 12,
 'm_orderRef': None,
 'm_permId': 656507451,
 'm_price': 2659.25,
 'm_shares': 9,
 'm_side': 'BOT',
 'm_time': '20180212  21:53:05'}

In [52]:
pd.DataFrame(callback.commission_Report.__dict__, index=[0])

,m_commission,m_currency,m_execId,m_realizedPNL,m_yield,m_yieldRedemptionDate
0,18.45,USD,0001f4e5.5a811f33.01.01,1.797693e+308,1.797693e+308,0


In [53]:
tws.eDisconnect()

***